In [10]:
# 0. SETTINGS (Windows-friendly)

import os

# 현재 작업 폴더 (mdm.ipynb가 있는 폴더)
ROOT = os.getcwd()

print("Project root:", ROOT)
print("HumanML3D folder:", ROOT)

# 모델이 한 번에 처리하는 최대 프레임 수
MAX_SEQ_LEN = 196

Project root: C:\Users\21600\Downloads\assignment\HumanML3D\HumanML3D
HumanML3D folder: C:\Users\21600\Downloads\assignment\HumanML3D\HumanML3D


In [6]:
# 1. IMPORTS (Windows compatibility)

import os
import random
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader

# 확인용 출력
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

PyTorch version: 1.13.1+cpu
CUDA available: False


In [12]:
# 2. DATASET & COLLATE

class HumanML3DDataset(Dataset):
    """
    HumanML3D 구조:
        Mean.npy, Std.npy
        train.txt / val.txt / test.txt
        new_joint_vecs/<id>.npy (motion: T, 263)
        texts/<id>.txt          (caption lines)
    """
    def __init__(self, data_root, split="train", normalize=True):
        super().__init__()
        self.data_root = data_root
        self.split = split
        self.normalize = normalize

        # split list 읽기
        split_file = os.path.join(data_root, f"{split}.txt")
        with open(split_file, "r") as f:
            ids = [line.strip().replace(".npy", "") for line in f.readlines()]
        self.sample_ids = ids

        # 모션 경로
        self.motion_dir = os.path.join(data_root, "new_joint_vecs")
        self.text_dir = os.path.join(data_root, "texts")

        # 정규화 (Mean, Std)
        if normalize:
            self.mean = np.load(os.path.join(data_root, "Mean.npy"))  # (263,)
            self.std = np.load(os.path.join(data_root, "Std.npy"))    # (263,)
        else:
            self.mean = None
            self.std = None

    def __len__(self):
        return len(self.sample_ids)

    def _load_motion(self, sid):
        path = os.path.join(self.motion_dir, sid + ".npy")
        motion = np.load(path).astype(np.float32)  # (T, 263)

        if self.normalize:
            motion = (motion - self.mean) / (self.std + 1e-8)

        return motion

    def _load_text(self, sid):
        path = os.path.join(self.text_dir, sid + ".txt")
        with open(path, "r", encoding="utf-8") as f:
            lines = [l.strip() for l in f.readlines() if l.strip()]
        # 문장이 여러 개라면 하나 랜덤 선택
        raw = random.choice(lines)
        caption = raw.split("#")[0].lower()
        return caption

    def __getitem__(self, idx):
        sid = self.sample_ids[idx]
        motion = self._load_motion(sid)
        caption = self._load_text(sid)

        return {
            "name": sid,
            "motion": motion,   # numpy (T, 263)
            "length": motion.shape[0],
            "text": caption
        }

In [13]:
# 2-2. COLLATE FUNCTION (Windows-safe)

def humanml_collate_fn(batch):
    # 길이 기준으로 정렬 (optional, RNN/Transformer에 안정적)
    batch = sorted(batch, key=lambda x: x["length"], reverse=True)

    lengths = [b["length"] for b in batch]
    max_len = max(lengths)

    motions = []
    texts = []
    names = []

    for b in batch:
        m = b["motion"]   # numpy (T, 263)
        T, D = m.shape

        # padding to max_len
        if T < max_len:
            pad = np.zeros((max_len - T, D), dtype=np.float32)
            m = np.concatenate([m, pad], axis=0)

        motions.append(torch.from_numpy(m))   # (max_len, 263)
        texts.append(b["text"])
        names.append(b["name"])

    motions = torch.stack(motions)   # (B, max_len, 263)
    lengths = torch.tensor(lengths, dtype=torch.long)

    return {
        "motion": motions,
        "length": lengths,
        "text": texts,
        "name": names
    }

In [14]:
# 2-3. DATALOADER TEST (Windows safe → num_workers=0)

train_dataset = HumanML3DDataset(DATA_ROOT, split="train")

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0,            # Windows: 반드시 0 !!!
    collate_fn=humanml_collate_fn,
    drop_last=True
)

batch = next(iter(train_loader))
print("motion shape:", batch["motion"].shape)   # (B, T, 263)
print("lengths:", batch["length"])
print("first text:", batch["text"][0])
print("first id:", batch["name"][0])

motion shape: torch.Size([4, 199, 263])
lengths: tensor([199,  84,  71,  56])
first text: the person is dancing the waltz.
first id: 011708


In [15]:
sample = train_dataset[0]
print("ID:", sample["name"])
print("Motion shape:", sample["motion"].shape)
print("Caption:", sample["text"])

batch = next(iter(train_loader))
print("Batch shape:", batch["motion"].shape)
print("Lengths:", batch["length"])

ID: 000001
Motion shape: (35, 263)
Caption: a man squats extraordinarily low then bolts up in an unsatisfactory jump.
Batch shape: torch.Size([4, 199, 263])
Lengths: tensor([199, 199, 145, 116])


In [16]:
print("Total samples:", len(train_dataset))
print("First 10 IDs:", train_dataset.sample_ids[:10])

Total samples: 23384
First 10 IDs: ['000001', '000002', '000003', '000004', '000005', '000006', '000007', '000008', '000009', '000010']


In [17]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

C:\Users\21600\miniconda3\envs\mdm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
class BERTTextEncoder(nn.Module):
    """
    HuggingFace BERT 기반 텍스트 인코더.
    입력: text list (배치 크기 B)
    출력: (B, D) 텍스트 임베딩
    """
    def __init__(self, model_name="bert-base-uncased", pooling="cls"):
        super().__init__()
        self.pooling = pooling

        # Load tokenizer & model
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.bert = BertModel.from_pretrained(model_name)

        # 출력 차원: 768
        self.embed_dim = self.bert.config.hidden_size

    def forward(self, text_list):
        """
        text_list: list[str], 길이가 서로 다른 텍스트들
        return: (B, D)
        """

        # Tokenize
        enc = self.tokenizer(
            text_list,
            padding=True,
            truncation=True,
            max_length=64,
            return_tensors="pt"
        ).to(next(self.bert.parameters()).device)

        out = self.bert(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"]
        )

        if self.pooling == "cls":
            # CLS 토큰 벡터
            text_emb = out.last_hidden_state[:, 0]  # (B, 768)
        else:
            # Attention mask 기반 mean pooling
            mask = enc["attention_mask"].unsqueeze(-1).float()  # (B, T, 1)
            sum_hidden = (out.last_hidden_state * mask).sum(dim=1)
            lengths = mask.sum(dim=1)
            text_emb = sum_hidden / lengths

        return text_emb  # (B, 768)

In [19]:
import torch.nn.functional as F

In [20]:
class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()

        self.attn = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.ff = nn.Sequential(
            nn.Linear(dim, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, dim)
        )
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)

        ff_out = self.ff(x)
        x = self.norm2(x + ff_out)
        return x


class MotionDiffusionTransformerConcat(nn.Module):
    def __init__(
        self,
        motion_dim=263,       # HumanML3D motion vector dimension
        text_dim=768,         # BERT embedding dim
        proj_dim=256,         # we project both into 256-d
        model_dim=512,        # Transformer hidden dimension
        num_layers=8,
        num_heads=8,
        ff_dim=1024,
        dropout=0.1
    ):
        super().__init__()

        # motion → 256
        self.motion_proj = nn.Linear(motion_dim, proj_dim)

        # text → 256
        self.text_proj = nn.Linear(text_dim, proj_dim)

        # concat → 512
        self.input_dim = proj_dim * 2

        # transformer backbone
        self.layers = nn.ModuleList([
            TransformerBlock(model_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])

        # to match 512 dim input if needed
        self.match_dim = nn.Linear(self.input_dim, model_dim)

        # final output projection (predict noise)
        self.output_proj = nn.Linear(model_dim, motion_dim)

    def forward(self, x, text_emb):
        """
        x: (B, T, motion_dim)
        text_emb: (B, text_dim)
        """
        B, T, _ = x.shape

        # project motion → (B, T, 256)
        motion_feat = self.motion_proj(x)

        # project text → (B, 256)
        text_feat = self.text_proj(text_emb)
        text_feat = text_feat[:, None, :].repeat(1, T, 1)  # (B, T, 256)

        # concat → (B, T, 512)
        h = torch.cat([motion_feat, text_feat], dim=-1)

        # match to transformer hidden dim (usually 512)
        h = self.match_dim(h)

        # transformer layers
        for layer in self.layers:
            h = layer(h)

        # predict noise
        out = self.output_proj(h)  # (B, T, 263)

        return out


In [21]:
def make_beta_schedule(num_steps=1000, start=1e-4, end=0.02):
    return torch.linspace(start, end, num_steps)


class DiffusionSchedule:
    def __init__(self, num_steps=1000, device="cpu"):
        self.num_steps = num_steps
        self.device = device
        
        self.betas = make_beta_schedule(num_steps).to(device)
        self.alphas = 1.0 - self.betas
        self.alpha_cumprod = torch.cumprod(self.alphas, dim=0)
        
        self.sqrt_alpha_cumprod = torch.sqrt(self.alpha_cumprod)
        self.sqrt_one_minus_alpha_cumprod = torch.sqrt(1.0 - self.alpha_cumprod)

    def extract(self, arr, t, x_shape):
        """
        arr: (num_steps,)
        t: (B,)
        return shape: (B, 1, 1)
        """
        return arr[t].view(-1, *([1] * (len(x_shape) - 1)))


# =========================
# q_sample: forward diffusion
# =========================

def q_sample(x0, t, schedule: DiffusionSchedule, noise=None):
    """
    x0: (B, T, D)
    t: (B,)
    """
    if noise is None:
        noise = torch.randn_like(x0)

    sqrt_alpha_cumprod_t = schedule.extract(schedule.sqrt_alpha_cumprod, t, x0.shape)
    sqrt_one_minus_alpha_cumprod_t = schedule.extract(schedule.sqrt_one_minus_alpha_cumprod, t, x0.shape)

    x_t = sqrt_alpha_cumprod_t * x0 + sqrt_one_minus_alpha_cumprod_t * noise
    return x_t, noise


# =========================
# Loss
# =========================

def diffusion_loss(model, schedule, x0, text_emb):
    B = x0.shape[0]
    device = x0.device

    t = torch.randint(0, schedule.num_steps, (B,), device=device)
    x_t, noise = q_sample(x0, t, schedule)

    eps_pred = model(x_t, text_emb)

    return F.mse_loss(eps_pred, noise)


# =========================
# Reverse Sampling (simple)
# =========================

@torch.no_grad()
def p_sample_simple(model, schedule, x_t, t, text_emb):
    """
    Simplified DDPM reverse step (no posterior coefficients)
    """
    eps_pred = model(x_t, text_emb)

    # Estimate x0
    sqrt_alpha_cumprod_t = schedule.extract(schedule.sqrt_alpha_cumprod, t, x_t.shape)
    sqrt_one_minus_alpha_cumprod_t = schedule.extract(schedule.sqrt_one_minus_alpha_cumprod, t, x_t.shape)

    x0_pred = (x_t - sqrt_one_minus_alpha_cumprod_t * eps_pred) / sqrt_alpha_cumprod_t

    # Compute x_{t-1}
    sqrt_alpha_cumprod_prev = schedule.sqrt_alpha_cumprod[t - 1].view(-1, 1, 1)
    sqrt_one_minus_alpha_cumprod_prev = schedule.sqrt_one_minus_alpha_cumprod[t - 1].view(-1, 1, 1)

    # random noise
    noise = torch.randn_like(x_t)

    # t == 0이면 noise 제거
    nonzero_mask = (t > 0).float().view(-1, 1, 1)

    x_prev = (
        sqrt_alpha_cumprod_prev * x0_pred +
        nonzero_mask * sqrt_one_minus_alpha_cumprod_prev * noise
    )

    return x_prev


@torch.no_grad()
def p_sample_loop_simple(model, schedule, shape, text_emb, device="cpu"):
    """
    Full reverse diffusion process (no posterior formula)
    """
    B = shape[0]
    x_t = torch.randn(shape, device=device)
    text_emb = text_emb.to(device)

    for step in reversed(range(schedule.num_steps)):
        t = torch.full((B,), step, device=device, dtype=torch.long)
        x_t = p_sample_simple(model, schedule, x_t, t, text_emb)

    return x_t

In [25]:
from tqdm import tqdm

def train(
    data_root,
    save_dir="./checkpoints",
    num_epochs=50,
    batch_size=16,
    lr=1e-4,
    num_diffusion_steps=1000,
    device=None,
    finetune_bert=False,
    num_workers=0,          # Windows이면 0 권장
    log_interval=50
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[INFO] Using device: {device}")

    os.makedirs(save_dir, exist_ok=True)

    # 1. Dataset & DataLoader
    train_dataset = HumanML3DDataset(
        data_root=data_root,
        split="train",
        normalize=True
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        collate_fn=humanml_collate_fn,
        drop_last=True,
    )

    # 2. Models
    text_encoder = BERTTextEncoder(
        model_name="bert-base-uncased",
        pooling="cls"
    ).to(device)

    motion_model = MotionDiffusionTransformerConcat(
        motion_dim=263,
        text_dim=text_encoder.embed_dim,
        proj_dim=256,
        model_dim=512,
        num_layers=8,
        num_heads=8,
        ff_dim=1024,
        dropout=0.1
    ).to(device)

    if not finetune_bert:
        for p in text_encoder.parameters():
            p.requires_grad = False
        text_encoder.eval()
        print("[INFO] BERT encoder is frozen (no finetuning).")
    else:
        print("[INFO] BERT encoder will be finetuned.")

    # 3. Diffusion schedule
    schedule = DiffusionSchedule(num_steps=num_diffusion_steps, device=device)

    # 4. Optimizer
    if finetune_bert:
        params = list(text_encoder.parameters()) + list(motion_model.parameters())
    else:
        params = list(motion_model.parameters())

    optimizer = torch.optim.AdamW(params, lr=lr)
    max_grad_norm = 1.0

    # 5. Training loop
    global_step = 0
    for epoch in range(1, num_epochs + 1):
        motion_model.train()
        if finetune_bert:
            text_encoder.train()
        else:
            text_encoder.eval()

        running_loss = 0.0

        # tqdm으로 진행률 + 실시간 loss 표시
        pbar = tqdm(
            enumerate(train_loader),
            total=len(train_loader),
            desc=f"Epoch {epoch}/{num_epochs}"
        )

        for batch_idx, batch in pbar:
            motions = batch["motion"].to(device)  # (B, T, 263)
            texts = batch["text"]                 # list[str]

            # 5-1. 텍스트 임베딩
            if finetune_bert:
                text_emb = text_encoder(texts)
            else:
                with torch.no_grad():
                    text_emb = text_encoder(texts)

            # 5-2. Diffusion loss 계산 (eps_pred vs noise)
            loss = diffusion_loss(
                model=motion_model,
                schedule=schedule,
                x0=motions,
                text_emb=text_emb
            )

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, max_grad_norm)
            optimizer.step()

            loss_val = loss.item()
            running_loss += loss_val
            global_step += 1

            # tqdm progress bar에 현재 loss 실시간 표시
            pbar.set_postfix({
                "step_loss": f"{loss_val:.6f}",
                "avg_loss": f"{running_loss / (batch_idx + 1):.6f}"
            })

            # 원래의 log_interval 출력도 유지하고 싶으면:
            if (batch_idx + 1) % log_interval == 0:
                avg_loss = running_loss / (batch_idx + 1)
                print(
                    f"[Epoch {epoch:03d}] "
                    f"Step {batch_idx+1:04d}/{len(train_loader):04d} | "
                    f"Global {global_step:06d} | "
                    f"step_loss: {loss_val:.6f} | "
                    f"avg_loss: {avg_loss:.6f}"
                )

        # 6. epoch마다 체크포인트 저장
        ckpt_path = os.path.join(save_dir, f"epoch_{epoch:03d}.pt")
        save_dict = {
            "epoch": epoch,
            "motion_model_state": motion_model.state_dict(),
            "text_encoder_state": text_encoder.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "config": {
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "lr": lr,
                "num_diffusion_steps": num_diffusion_steps,
                "finetune_bert": finetune_bert,
            }
        }
        torch.save(save_dict, ckpt_path)
        print(f"[INFO] Saved checkpoint: {ckpt_path}")

In [26]:
# ─────────────────────────────
# MAIN ENTRY
# ─────────────────────────────
if __name__ == "__main__":
    # HumanML3D가 ROOT 바로 아래에 있다고 가정
    data_root = ROOT

    train(
        data_root=data_root,
        save_dir=os.path.join(ROOT, "checkpoints_humanml"),
        num_epochs=50,
        batch_size=16,
        lr=1e-4,
        num_diffusion_steps=1000,
        finetune_bert=False,      # 처음엔 False로 두고 실험 추천
        num_workers=0,            # Windows면 0
        log_interval=50
    )

[INFO] Using device: cpu


C:\Users\21600\miniconda3\envs\mdm\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly 

[INFO] BERT encoder is frozen (no finetuning).


Epoch 1/50:   3%|▉                            | 50/1461 [01:12<33:37,  1.43s/it, step_loss=0.953116, avg_loss=1.007780]

[Epoch 001] Step 0050/1461 | Global 000050 | step_loss: 0.953116 | avg_loss: 1.007780


...

[INFO] Saved checkpoint: C:\Users\21600\Downloads\assignment\HumanML3D\HumanML3D\checkpoints_humanml\epoch_034.pt


Epoch 35/50:   3%|▉                           | 50/1461 [01:06<30:34,  1.30s/it, step_loss=0.067410, avg_loss=0.050719]

[Epoch 035] Step 0050/1461 | Global 049724 | step_loss: 0.067410 | avg_loss: 0.050719


Epoch 35/50:   6%|█▋                          | 89/1461 [01:56<29:58,  1.31s/it, step_loss=0.038006, avg_loss=0.050401]


KeyboardInterrupt: 

In [27]:
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)
print(parent_dir)

C:\Users\21600\Downloads\assignment\HumanML3D


In [9]:
#BVH Conversion code
from skeleton import Skeleton
t2m_raw_offsets = np.array([[0,0,0],
                           [1,0,0],
                           [-1,0,0],
                           [0,1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,1,0],
                           [0,0,1],
                           [0,0,1],
                           [0,1,0],
                           [1,0,0],
                           [-1,0,0],
                           [0,0,1],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0]])

t2m_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

def recover_root_rot_pos(feats_263):
    # feats_263: (1,T,263)
    root_vel = feats_263[..., :2]              # (1,T,2)
    root_y   = feats_263[..., 2:3]             # (1,T,1)
    root_quat = feats_263[..., 3:7]            # (1,T,4)

    traj = torch.cumsum(root_vel, dim=1)
    root_pos = torch.cat([traj, root_y], dim=-1)[0]  # (T,3)

    return root_quat, root_pos
    
def quaternion_to_cont6d(quat):
    # quat: (1,T,4)  xyzw
    rot = R.from_quat(quat[0].cpu().numpy())
    mats = rot.as_matrix()   # (T,3,3)
    mats = torch.from_numpy(mats).float()

    a1 = mats[..., :3, 0]
    a2 = mats[..., :3, 1]
    return torch.stack([a1, a2], dim=-1).reshape(1, mats.shape[0], 6)

def extract_rot6d_all_from_263(feats_263):
    B,T,_ = feats_263.shape
    J = 22

    root_quat, root_pos = recover_root_rot_pos(feats_263)
    root_6d = quaternion_to_cont6d(root_quat)   # (1,T,6)

    start = 1 + 2 + 1 + (J-1)*3     # = 67
    end   = start + (J-1)*6         # = 193

    body = feats_263[..., start:end].view(1,T,J-1,6)
    rot6d_all = torch.cat([root_6d.unsqueeze(2), body], dim=2)  # (1,T,J,6)

    return root_pos, rot6d_all[0]

def cont6d_to_matrix(rot6d):
    a1 = rot6d[..., 0:3]
    a2 = rot6d[..., 3:6]
    b1 = torch.nn.functional.normalize(a1, dim=-1)
    proj = (b1 * a2).sum(dim=-1, keepdim=True) * b1
    b2 = torch.nn.functional.normalize(a2 - proj, dim=-1)
    b3 = torch.cross(b1, b2, dim=-1)
    return torch.stack([b1, b2, b3], dim=-1)

def rot6d_to_euler_deg(rot6d, order="ZXY"):
    T,J,_ = rot6d.shape
    mats = cont6d_to_matrix(rot6d).reshape(T*J,3,3).cpu().numpy()
    euler = R.from_matrix(mats).as_euler(order, degrees=True)
    return euler.reshape(T,J,3)

def write_bvh_hierarchy(f, skel, names):
    parents = skel.parents()
    offsets = skel.offset().cpu().numpy()
    
    children = [[] for _ in parents]
    for i,p in enumerate(parents):
        if p >= 0: children[p].append(i)

    def rec(j, depth):
        ind = "  "*depth
        nm = names[j]
        ox,oy,oz = offsets[j]

        if parents[j] == -1:
            f.write(f"ROOT {nm}\n")
        else:
            f.write(f"{ind}JOINT {nm}\n")

        f.write(f"{ind}{{\n")
        f.write(f"{ind}  OFFSET {ox:.6f} {oy:.6f} {oz:.6f}\n")

        if parents[j] == -1:
            f.write(f"{ind}  CHANNELS 6 Xposition Yposition Zposition Zrotation Xrotation Yrotation\n")
        else:
            f.write(f"{ind}  CHANNELS 3 Zrotation Xrotation Yrotation\n")

        for c in children[j]: rec(c, depth+1)
        f.write(f"{ind}}}\n")

    f.write("HIERARCHY\n")
    rec(0,0)

def write_bvh_motion(f, root_pos, euler, parents, fps=20):
    T,J,_ = euler.shape
    rp = root_pos.cpu().numpy()

    f.write("MOTION\n")
    f.write(f"Frames: {T}\n")
    f.write(f"Frame Time: {1/fps:.8f}\n")

    for t in range(T):
        vals = []
        for j in range(J):
            z,x,y = euler[t,j]
            if parents[j] == -1:
                px,py,pz = rp[t]
                vals += [px,py,pz,z,x,y]
            else:
                vals += [z,x,y]
        f.write(" ".join(f"{v:.6f}" for v in vals)+"\n")

def motion263_to_bvh(motion_263, mean, std, save_path, device="cpu"):
    device = torch.device(device)

    if isinstance(motion_263, np.ndarray):
        motion_263 = torch.from_numpy(motion_263)
    if motion_263.dim() == 2:
        motion_263 = motion_263.unsqueeze(0)

    mean_t = torch.from_numpy(mean).view(1,1,-1)
    std_t  = torch.from_numpy(std).view(1,1,-1)
    feats  = motion_263 * std_t + mean_t   # denorm

    # Extract motion
    root_pos, rot6d_all = extract_rot6d_all_from_263(feats.to(device))

    # Create skeleton
    raw = torch.tensor(t2m_raw_offsets, dtype=torch.float32)
    skel = Skeleton(raw, t2m_kinematic_chain, device)

    # Use FK to compute joint locations → offsets computed from first frame
    joints = skel.forward_kinematics_cont6d(rot6d_all.unsqueeze(0), root_pos.unsqueeze(0))[0]
    skel.get_offsets_joints(joints[0])   # set offset

    # Convert rotation to Euler
    euler = rot6d_to_euler_deg(rot6d_all)

    parents = skel.parents()
    names = [f"J{i}" for i in range(22)]

    with open(save_path, "w") as f:
        write_bvh_hierarchy(f, skel, names)
        write_bvh_motion(f, root_pos, euler, parents)

    print("Saved BVH:", save_path)